In [0]:
%fs ls /mnt/anac/Silver

In [0]:
df = spark.read.parquet("dbfs:/mnt/anac/Silver/V_OCORRENCIA_AMPLA.parquet/")
display(df)

In [0]:
colunas = ['Aerodromo_de_Destino', 'Aerodromo_de_Origem', 'Categoria_da_Aeronave', 'Classificacao_da_Ocorrência', 'Danos_a_Aeronave', 'Data_da_Ocorrencia', 'Descricao_do_Tipo', 'Fase_da_Operacao', 'Historico', 'Hora_da_Ocorrência', 'Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes', 'Modelo', 'Municipio', 'Nome_do_Fabricante', 'UF']

In [0]:
df = df.select(colunas)
display(df)

In [0]:
colunas_somatoria = ['Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']

df = df.withColumn('Total_Lesoes', sum(df[coluna] for coluna in colunas_somatoria))
display(df)

In [0]:
df = df.withColumnRenamed('Aerodromo_de_Destino',  'Destino') \
       .withColumnRenamed('Aerodromo_de_Origem', 'Origem') \
       .withColumnRenamed('Categoria_da_Aeronave', 'Categoria') \
       .withColumnRenamed('Classificacao_da_Ocorrência', 'Classificacao') \
       .withColumnRenamed('Danos_a_Aeronave', 'Danos') \
       .withColumnRenamed('Data_da_Ocorrencia', 'Data') \
       .withColumnRenamed('Descricao_do_Tipo', 'Descricao') \
       .withColumnRenamed('Fase_da_Operacao', 'Fase') \
       .withColumnRenamed('Historico', 'Historico') \
       .withColumnRenamed('Hora_da_Ocorrência', 'Hora') \
       .withColumnRenamed('UF', 'Estado')

display(df)

In [0]:
# ~  = funciona como negação.
classificacoes_excluir = ['Indeterminado', 'Sem registro', 'Exterior']
display(df.filter(~df.Estado.isin(classificacoes_excluir)))
df = df.filter(~df.Estado.isin(classificacoes_excluir))
display(df)

In [0]:
from pyspark.sql.functions import current_timestamp, date_format, from_utc_timestamp
df = df.withColumn('Atualizacao', date_format(from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"), "yyyy-MM-dd HH:mm:ss"))
display(df)

In [0]:
df.write \
  .mode("overwrite") \
  .partitionBy("Estado") \
  .parquet("dbfs:/mnt/anac/Gold/V_OCORRENCIA_AMPLA_FORMATADO.parquet/")